In [6]:
from gpt import GPTConfig, GPT, generate
import torch
import os
from transformers import AutoTokenizer


In [11]:
def print_leaderboard_and_get_best_model():
    candidates = []
    for expt in os.listdir('logs'):
        if 'smoke' in expt:
            continue
        best_loss_for_expt = 10000
        best_line = ''

        for line in open(f'logs/{expt}/log.txt').readlines():

            if 'val' in line and 'byte loss' in line:
                def get_byte_loss():
                    after_bl_str = line.split('byte loss')[1]            
                    try:                
                        byte_loss = float(after_bl_str.split(' ')[1])
                    except ValueError as e:
                        byte_loss = 100000
                    return byte_loss
                
                byte_loss = get_byte_loss()

                if byte_loss < best_loss_for_expt:
                    best_loss_for_expt = byte_loss
                    best_line = line
        
        if best_loss_for_expt < 10000:
            candidates.append((best_loss_for_expt, expt, best_line))

    candidates = sorted(candidates, key=lambda x: x[0])
    for best_loss, expt, line in candidates:
        print(expt.ljust(50), f'{best_loss:.3} {line.strip()}')

    best_expt = candidates[0][1]
    best_model_paths = [model_path for model_path in os.listdir(f'logs/{best_expt}') if 'model' in model_path]
    best_model_path = sorted(best_model_paths, key=lambda x: x.split('_')[1])[-1]
    return best_expt, best_model_path

best_expt, best_model_path = print_leaderboard_and_get_best_model()


16l_16h_512d_hour_good_max                         0.31 step 31750 | val loss 1.2668 | byte loss 0.3097 | ds 3492.7s
16l_16h_512d_hour                                  0.325 step 33523 | val loss 1.3311 | byte loss 0.3254 | ds 4448.7s
16l_16h_512d_8_hour_good_max                       0.344 step 66750 | val loss 1.4055 | byte loss 0.3436 | ds 7307.2s
16l_16h_512d_quick                                 0.85 step 499 | val loss 3.4753 | byte loss 0.8496 | ds 84.3s


In [12]:
choosen_model = f'logs/{best_expt}/{best_model_path}'
# choosen_model = f'logs/16l_16h_512d_quick/model_00323.pt'
full_checkpoint = torch.load(choosen_model)
config = full_checkpoint['config']
m = GPT(config)

def remove_orig_mod_prefix(state_dict):
    return {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}

m.load_state_dict(remove_orig_mod_prefix(full_checkpoint['model']))
m.to('cuda')

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(8192, 512)
    (wpe): Embedding(512, 512)
    (h): ModuleList(
      (0-15): 16 x Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=512, out_features=1536, bias=True)
          (c_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=1024, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=1024, out_features=512, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=8192, bias=False)
)

In [9]:
enc = AutoTokenizer.from_pretrained('activated-ai/tiny-stories-8k-tokenizer')

In [13]:
generate(m, enc, "Lily went to the park and saw a friendly dog.", 255, 4)

sample 0: Lily went to the park and saw a friendly dog. The dog barked and Lily went to pet him. The dog's owner said, "Do you want to ride my pony?" Lily was so happy! She climbed on the pony and they rode around the park. But then, the pony suddenly stopped. Lily was scared and didn't know what to do. 

The owner said, "Don't worry, I can reverse the pony. Can you hold the handle again?" Lily held the handle and the horses started moving again. Lily was so happy and said, "Thank you for your help, my pony is his best friend forever!" They all went home and Lily told her mommy all about her pony ride.
sample 1: Lily went to the park and saw a friendly dog. She gave the dog a hug and said, "Hi, dog! What's your name?"

The dog wagged its tail and barked, "Woof! My name is Max! Do you want to play with me?" Lily and Max played, running and jumping in the grass. They had so much fun.

Then, Lily remembered that she had to go home soon. She said goodbye to Max and ran home. When she got h